<img src="https://github.com/pmservice/ai-openscale-tutorials/raw/master/notebooks/images/banner.png" align="left" alt="banner">

# IBM Watson OpenScale: Model Deployment and Monitoring (Simplified)

This notebook deploys a German Credit Risk model to Watson Machine Learning (WML), onboards it to Watson OpenScale (WOS), configures monitors (quality, fairness, drift, explainability), and injects historical scoring and feedback data.

> **Runtime Requirement**: IBM Runtime 24.1 (Python 3.11 XS)  
> **Services Required**: Watson Machine Learning, Watson OpenScale, Cloud Object Storage  
> **Optional**: Db2 Warehouse or Databases for PostgreSQL for external datamart

Ensure all required credentials and configuration values are defined before running the cell below.


### Package installation


In [ ]:
# !rm -rf /home/spark/shared/user-libs/python3.7*

# If you are executing this notebook in non IBM Watson Studio env then uncomment the pip install statements
#!pip install --upgrade scikit-learn==1.1   
# !pip install --upgrade pandas==0.25.3 
# !pip install --upgrade requests==2.23 
# !pip install --upgrade numpy==1.20.3 
# !pip install --upgrade SciPy 

!pip install --upgrade ibm-watson-machine-learning
!pip install --upgrade "ibm-watson-openscale~=3.0.34" 

!rm german_credit_data_biased_training.csv
!rm german_credit_feedback_data.json
!rm german_credit_scoring_data.json

!wget https://raw.githubusercontent.com/IBM/watson-openscale-samples/main/IBM%20Cloud/WML/assets/data/credit_risk/german_credit_data_biased_training.csv -O german_credit_data_biased_training.csv
!wget https://raw.githubusercontent.com/IBM/watson-openscale-samples/main/IBM%20Cloud/WML/assets/data/credit_risk/german_credit_feedback_data.json -O german_credit_feedback_data.json
!wget https://raw.githubusercontent.com/IBM/watson-openscale-samples/main/IBM%20Cloud/WML/assets/data/credit_risk/german_credit_scoring_data.json -O german_credit_scoring_data.json


## Step 1: Define Credentials and Basic Configuration




## Configuration Dictionary Inputs

The following table describes the inputs expected in the `config_dict` used to deploy and monitor a model. Some inputs are required, while others are optional and will default to predefined values if not provided.

| Input Name                  | Expected Type/Format     | Required/Optional | Default Value (if any)         | Description |
|----------------------------|--------------------------|-------------------|--------------------------------|-------------|
| `wml_space_id`             | `str` (UUID)             | Required          | –                              | ID of the Watson Machine Learning deployment space. |
| `cloud_api_key`            | `str`                    | Required          | –                              | IBM Cloud API key used for authentication. |
| `wml_url`                  | `str` (URL)              | Optional          | `"https://us-south.ml.cloud.ibm.com"` | URL of the Watson Machine Learning service. |
| `bucket_name`              | `str`                    | Required          | –                              | Name of the Cloud Object Storage bucket. |
| `cos_resource_crn`         | `str`                    | Required          | –                              | Cloud Object Storage resource CRN. |
| `cos_endpoint`             | `str` (URL)              | Required          | –                              | Endpoint URL for the COS instance. |
| `cos_api_key_id`           | `str`                    | Required          | –                              | API key ID for accessing COS. |
| `cos_api_key`              | `str`                    | Required          | –                              | API key for accessing COS . |
| `db_credentials`           | `dict` or `None`         | Optional          | `None`                           | External database credentials (if used). |
| `schema_name`              | `str` or `None`          | Optional          | `None`                     | Schema name for external database. |
| `training_data_df`         | `pandas.DataFrame`       | Required          | –                              | DataFrame containing the training data. |
| `training_data_file_name`  | `str` (filename)         | Required          | –                              | Name of the training data file to upload to COS. |
| `model_object`             | Trained model object     | Required          | –                              | Trained model (e.g., scikit-learn pipeline) to be deployed. |
| `service_provider_name`    | `str`                    | Optional          | `"Watson Machine Learning V1"` | Name of the model provider. |
| `service_provider_description` | `str`               | Optional          | `"Added by tutorial WOS notebook."` | Description of the model provider. |
| `model_name`               | `str`                    | Optional          | `"Scikit German Risk Model WML V1"` | Name of the model asset. |
| `deployment_name`          | `str`                    | Optional          | `"Scikit German Risk Deployment WML V1"` | Name of the model deployment. |
| `asset_name`               | `str`                    | Optional          | `"Scikit German Risk Model WML V1"`          | Name of the model asset in OpenScale. |
| `label_field`              | `str`                    | Required          | -                      | Target column in the dataset. |
| `prediction_field`         | `str`                    | Required          | -                 | Output field for predictions. |
| `probability_fields`       | `list[str]`              | Required          | -            | List of probability output fields. |
| `feature_fields`           | `list[str]`              | Required          | -  | Input features used by the model. |
| `categorical_fields`       | `list[str]`              | Required          | -     | Categorical input features. |
| `sample_data_fields`       | `list[str]`              | Required          | -      | Field names for sample scoring input. |
| `sample_data_values`       | `list[list]`             | Required          | -          | Sample records for scoring simulation. |
| `feedback_data_file`       | `str` (filename)         | Required          | - | File containing historical feedback records. |
| `scoring_data_file`        | `str` (filename)         | Required          | -` | File containing historical scoring requests. |
| `quality_monitor_params`   | `dict`                   | Required          | - | Parameters for quality monitor. |
| `quality_monitor_thr`      | `list[dict]`             | Required          | -       | Threshold for triggering quality alerts. |
| `fairness_monitor_params`  | `dict`                   | Required          | -    | Configuration for fairness monitor. |
| `fairness_monitor_thr`     | `None` or `list[dict]`   | Required          | –                          | Threshold for fairness alerts (if any). |
| `explainability_params`    | `dict`                   | Required          | -        | Enable or disable explainability. |
| `driftv2_params`           | `dict`                   | Required          | -       | Drift monitor configuration. |
| `problem_type`             | `str`                    | Optional          | `"binary"`                     | Type of machine learning problem. Supported values include `"binary"`, `"multiclass"`, and `"regression"`.                           | Type of ML problem (e.g., binary, multiclass, regression). |
| `input_data_type`          | `str`                    | Optional          | `"structured"`                 | Type of input data. Supported values include `"structured"`, `"unstructured_image"`, `"unstructured_text"`, `"unstructured_audio"`, etc.                        | Type of input data (e.g., structured, unstructured). |
| `force_deployment_recreate`           | `bool`                   | Optional          | `False`         | Controls the Watson Machine Learning (WML) deployment. <br> - If `False`: The API will **reuse** an existing WML deployment if `deployment_name` is found. If not found, a new deployment is created.<br> - If `True`: The API will **delete** any existing WML deployment with `deployment_name` and then **create a new one** (resulting in a new internal Deployment ID).  |
| `monitor_new_version`           | `bool`                   | Optional          | `False`        |  Controls the Watson OpenScale monitoring subscription. <br> - If `False`: The API will **reuse** an existing OpenScale subscription if one is found for the current WML Deployment ID. If not found, a new subscription is created.<br> - If `True`: The API will **always create a new** OpenScale subscription, even if one already exists for the WML Deployment ID. |


### How force_deployment_recreate and monitor_new_version Combinations Affect Behavior:



The combination of these flags dictates the API's overall action:



1. **Default/Continue Run:**

* `force_deployment_recreate = False`

* `monitor_new_version = False`

* **Behavior:** Reuses existing WML Deployment and OpenScale Subscription if found. Performs initial data log and monitor run. 



2. **New Endpoint & New Monitoring:**

* `force_deployment_recreate = False`

* `monitor_new_version = True`

* **Behavior:** Creates a **new WML Deployment** (requires a new `deployment_name` input from the user). Creates a **new OpenScale Subscription**. Performs initial data log and monitor run for the new setup. 



3. **Overwrite Endpoint & New Monitoring:**

* `force_deployment_recreate = True`

* `monitor_new_version = True`

* **Behavior:** **Deletes** the existing WML Deployment and **creates a new one** with the same name. Creates a **new OpenScale Subscription**. Performs initial data log and monitor run for the new setup. 




Reference: Optional Parameter Configuration

For detailed steps on how to set and configure the optional parameters, please refer to the following notebook:

[Watson OpenScale and Watson ML Engine - scikit-learn Binary Classification Example](https://github.com/IBM/watson-openscale-samples/blob/main/IBM%20Cloud/WML/notebooks/binary/scikit_learn/Watson%20OpenScale%20and%20Watson%20ML%20Engine.ipynb)





In [ ]:
WML_URL = "https://us-south.ml.cloud.ibm.com"
CLOUD_API_KEY = "<EDIT THIS>"
SPACE_ID = "<EDIT THIS>"


TRAINING_DATA_FILE_NAME = "<EDIT THIS>" #eg "german_credit_data_biased_training.csv
COS_API_KEY_ID = "<EDIT THIS>"
COS_API_KEY = "<EDIT THIS>"
COS_RESOURCE_CRN = "<EDIT THIS>" # eg "crn:v1:bluemix:public:cloud-object-storage:global:a/3bf0d9003abfb5d29761c3e97696b71c:d6f04d83-6c4f-4a62-a165-696756d63903::"
COS_ENDPOINT = "<EDIT THIS>" # eg"https://s3.us-south.cloud-object-storage.appdomain.cloud"
BUCKET_NAME = "<EDIT THIS>"



## Step 2: Load and Upload Training Data

This section performs two key tasks:

1. **Load the training dataset** from a local CSV file into a Pandas DataFrame.
2. **Upload the dataset to Cloud Object Storage (COS)** so it can be referenced during model deployment and monitoring.


Ensure that the COS credentials and bucket name are correctly defined before running this step.


In [2]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.linear_model import SGDClassifier
from sklearn.pipeline import Pipeline
from sklearn.metrics import roc_auc_score
from sklearn.inspection import permutation_importance
import ibm_boto3
from ibm_botocore.client import Config


training_data_df = pd.read_csv(TRAINING_DATA_FILE_NAME)

cos_client = ibm_boto3.resource(
    "s3",
    ibm_api_key_id=CLOUD_API_KEY,
    ibm_service_instance_id=COS_RESOURCE_CRN,
    ibm_auth_endpoint="https://iam.bluemix.net/oidc/token",
    config=Config(signature_version="oauth"),
    endpoint_url=COS_ENDPOINT
)

with open(TRAINING_DATA_FILE_NAME, "rb") as file_data:
    cos_client.Object(BUCKET_NAME, TRAINING_DATA_FILE_NAME).upload_fileobj(Fileobj=file_data)


## Step 3: Train and Evaluate the Model

This section prepares and trains a machine learning pipeline using the German Credit Risk dataset. It also evaluates the model's performance using AUC (Area Under the ROC Curve).

The trained model is stored in `MODEL_OBJECT` and the full dataset in `DATA_DF` for later use in deployment and monitoring.


In [3]:
# Train model
train_data, test_data = train_test_split(training_data_df, test_size=0.2)
features_idx = np.s_[0:-1]
first_record_idx = np.s_[0]
string_fields = [type(fld) is str for fld in train_data.iloc[first_record_idx, features_idx]]

ct = ColumnTransformer([
    ("ohe", OneHotEncoder(), list(np.array(train_data.columns)[features_idx][string_fields]))
])

clf_linear = SGDClassifier(loss='log_loss', penalty='l2', max_iter=1000, tol=1e-5)

pipeline_linear = Pipeline([
    ('ct', ct), 
    ('clf_linear', clf_linear)
])

risk_model = pipeline_linear.fit(train_data.drop('Risk', axis=1), train_data.Risk)
MODEL_OBJECT = risk_model
TRAINING_DATA_DF = training_data_df

# Evaluate model
predictions = risk_model.predict(test_data.drop('Risk', axis=1))
indexed_preds = [0 if prediction == 'No Risk' else 1 for prediction in predictions]
real_observations = test_data.Risk.replace('Risk', 1).replace('No Risk', 0).values
auc = roc_auc_score(real_observations, indexed_preds)
print("Model AUC:", auc)


Model AUC: 0.7643576290414067


In [4]:

WML_URL = "https://us-south.ml.cloud.ibm.com"

# Metadata
LABEL_FIELD = "Risk"
# Fields
PREDICTION_FIELD = "prediction"
PROBABILITY_FIELDS = ["probability"]
FEATURE_FIELDS = [
    "CheckingStatus",
    "LoanDuration",
    "CreditHistory",
    "LoanPurpose",
    "LoanAmount",
    "ExistingSavings",
    "EmploymentDuration",
    "InstallmentPercent",
    "Sex",
    "OthersOnLoan",
    "CurrentResidenceDuration",
    "OwnsProperty",
    "Age",
    "InstallmentPlans",
    "Housing",
    "ExistingCreditsCount",
    "Job",
    "Dependents",
    "Telephone",
    "ForeignWorker",
]
CATEGORICAL_FIELDS = [
    "CheckingStatus",
    "CreditHistory",
    "LoanPurpose",
    "ExistingSavings",
    "EmploymentDuration",
    "Sex",
    "OthersOnLoan",
    "OwnsProperty",
    "InstallmentPlans",
    "Housing",
    "Job",
    "Telephone",
    "ForeignWorker",
]

# Sample scoring data
SAMPLE_DATA_FIELDS = FEATURE_FIELDS
SAMPLE_DATA_VALUES = [
    [
        "no_checking",
        13,
        "credits_paid_to_date",
        "car_new",
        1343,
        "100_to_500",
        "1_to_4",
        2,
        "female",
        "none",
        3,
        "savings_insurance",
        46,
        "none",
        "own",
        2,
        "skilled",
        1,
        "none",
        "yes",
    ],
    [
        "no_checking",
        24,
        "prior_payments_delayed",
        "furniture",
        4567,
        "500_to_1000",
        "1_to_4",
        4,
        "male",
        "none",
        4,
        "savings_insurance",
        36,
        "none",
        "free",
        2,
        "management_self-employed",
        1,
        "none",
        "yes",
    ],
]

# Files
FEEDBACK_DATA_FILE = "german_credit_feedback_data.json"
SCORING_DATA_FILE = "german_credit_scoring_data.json"

# Monitor configurations
QUALITY_MONITOR_PARAMS = {"min_feedback_data_size": 50}
QUALITY_MONITOR_THR = [
    {"metric_id": "area_under_roc", "type": "lower_limit", "value": 0.80}
]
FAIRNESS_MONITOR_PARAMS = {
    "features": [
        {
            "feature": "Sex",
            "majority": ["male"],
            "minority": ["female"],
            "threshold": 0.95,
        },
        {
            "feature": "Age",
            "majority": [[26, 75]],
            "minority": [[18, 25]],
            "threshold": 0.95,
        },
    ],
    "favourable_class": ["No Risk"],
    "unfavourable_class": ["Risk"],
    "min_records": 100,
}
FAIRNESS_MONITOR_THR = None
EXPLAINABILITY_PARAMS = {"enabled": True}
DRIFTV2_FEATURE_IMPORTANCE = {
    "CheckingStatus": 0.027799999999999936,
    "LoanDuration": 0.0,
    "CreditHistory": 0.005449999999999977,
    "LoanPurpose": 0.0023499999999999632,
    "LoanAmount": 0.0,
    "ExistingSavings": 0.0007500000000000284,
    "EmploymentDuration": 0.04099999999999997,
    "InstallmentPercent": 0.0,
    "Sex": 0.014049999999999962,
    "OthersOnLoan": 0.01044999999999996,
    "CurrentResidenceDuration": 0.0,
    "OwnsProperty": 0.03159999999999994,
    "Age": 0.0,
    "InstallmentPlans": 0.0023999999999999577,
    "Housing": 0.0007000000000000117,
    "ExistingCreditsCount": 0.0,
    "Job": 0.003449999999999953,
    "Dependents": 0.0,
    "Telephone": 0.005949999999999967,
    "ForeignWorker": 0.0005499999999999394,
}
DRIFTV2_PARAMS = {
    "min_samples": 10,
    "max_samples": 1000,
    "train_archive": True,
    "features": {"fields": FEATURE_FIELDS, "importances": DRIFTV2_FEATURE_IMPORTANCE},
}



## Step 4: Configure and Run Deployment & Monitoring

This section defines a single configuration dictionary (`config_dict`) that consolidates all required inputs for deploying the model, onboarding it to Watson OpenScale, and enabling monitors.


Once the configuration is complete, the `deploy_and_monitor_model()` function is called to:
1. Deploy the model to Watson Machine Learning.
2. Onboard it to Watson OpenScale.
3. Configure the selected monitors.
4. Inject historical scoring and feedback data.
5. Run the monitors.



In [5]:
config_dict = {
    "wml_space_id": SPACE_ID,
    "bucket_name": BUCKET_NAME,
    "cloud_api_key": CLOUD_API_KEY,
    "cos_resource_crn": COS_RESOURCE_CRN,
    "cos_endpoint": COS_ENDPOINT,
    "cos_api_key_id": COS_API_KEY_ID,
    "cos_api_key": CLOUD_API_KEY,
    "training_data_df": TRAINING_DATA_DF,
    "training_data_file_name": TRAINING_DATA_FILE_NAME,
    "model_object": MODEL_OBJECT,
    "wml_url": WML_URL,
    "label_field": LABEL_FIELD,
    "prediction_field": PREDICTION_FIELD,
    "probability_fields": PROBABILITY_FIELDS,
    "feature_fields": FEATURE_FIELDS,
    "categorical_fields": CATEGORICAL_FIELDS,
    "sample_data_fields": SAMPLE_DATA_FIELDS,
    "sample_data_values": SAMPLE_DATA_VALUES,
    "feedback_data_file": FEEDBACK_DATA_FILE,
    "scoring_data_file": SCORING_DATA_FILE,
    "quality_monitor_params": QUALITY_MONITOR_PARAMS,
    "quality_monitor_thr": QUALITY_MONITOR_THR,
    "fairness_monitor_params": FAIRNESS_MONITOR_PARAMS,
    "fairness_monitor_thr": FAIRNESS_MONITOR_THR,
    "explainability_params": EXPLAINABILITY_PARAMS,
    "driftv2_params": DRIFTV2_PARAMS,
    # "force_deployment_recreate": FORCE_DEPLOYMENT_RECREATE,
    # "monitor_new_version": MONITOR_NEW_VERSION,
    # "problem_type": PROBLEM_TYPE,
    # "input_data_type": INPUT_DATA_TYPE,
    # "model_name": MODEL_NAME,
    # "deployment_name": DEPLOYMENT_NAME,    
    # "asset_name": ASSET_NAME,
    # "db_credentials": DB_CREDENTIALS,
    # "schema_name": SCHEMA_NAME,
    # "service_provider_name": SERVICE_PROVIDER_NAME,
    # "service_provider_description": SERVICE_PROVIDER_DESCRIPTION,    
}


# Execute deployment and monitoring pipeline
from ibm_watson_openscale.notebooks_simplification.predictive_models_simplification import deploy_and_monitor_model
deploy_and_monitor_model(config=config_dict)


All configuration inputs are valid and accessible.
Creating connections...
SUCCESS
Software Specification ID: 45f12dfe-aa78-5b8d-9f38-0ee223c47309
Storing model to WML ...
Storing model Done
Model ID: 6334a159-3afb-4941-8ca5-5190873cc735
Deploying model...


#######################################################################################

Synchronous deployment creation for uid: '6334a159-3afb-4941-8ca5-5190873cc735' started

#######################################################################################


initializing
Note: online_url and serving_urls are deprecated and will be removed in a future release. Use inference instead.

ready


------------------------------------------------------------------------------------------------
Successfully finished deployment creation, deployment_uid='cecbef63-6f82-407b-b600-0398b822b4b8'
------------------------------------------------------------------------------------------------


Deploying model completed successfully;  depl

WOS Data Mart,Data Mart created by WOS tutorial notebook,True,active,2025-06-29 03:58:59.694000+00:00,bf3506fb-a335-4b00-8c81-0b348b5a93f4


Using existing datamart bf3506fb-a335-4b00-8c81-0b348b5a93f4
Configuring service provider
Configuring subscription



 Waiting for end of adding subscription 0197c9c8-ce12-7529-9800-2852822793a2 




active

-------------------------------------------
 Successfully finished adding subscription 
-------------------------------------------


New subscription created successfully with the ID: 0197c9c8-ce12-7529-9800-2852822793a2
Payload data set id: 0197c9c8-d823-7d63-a751-407833cf6523
Scoring the model to enable monitoring
scoring the model is completed
verifying that the payload logging records
Payload logging did not happen, performing explicit payload logging.
Number of records in the payload logging table: 20
Number of records in the payload logging table: 20
Creating quality monitor



 Waiting for end of monitor instance creation 0197c9c9-236c-735e-88e2-1eea0acc4032 




preparing
active

---------------------------------------
 Monitor instance successfully created 
--------------

2025-07-02 06:18:31.146000+00:00,true_positive_rate,0197c9c9-6f6a-79ec-ac1d-a72c8a14a626,0.6363636363636364,None,None,['model_type:original'],quality,0197c9c9-236c-735e-88e2-1eea0acc4032,a2e0d5ba-f559-4afc-b1cd-b70a0bb90efa,subscription,0197c9c8-ce12-7529-9800-2852822793a2
2025-07-02 06:18:31.146000+00:00,area_under_roc,0197c9c9-6f6a-79ec-ac1d-a72c8a14a626,0.7489510489510489,0.8,None,['model_type:original'],quality,0197c9c9-236c-735e-88e2-1eea0acc4032,a2e0d5ba-f559-4afc-b1cd-b70a0bb90efa,subscription,0197c9c8-ce12-7529-9800-2852822793a2
2025-07-02 06:18:31.146000+00:00,precision,0197c9c9-6f6a-79ec-ac1d-a72c8a14a626,0.7,None,None,['model_type:original'],quality,0197c9c9-236c-735e-88e2-1eea0acc4032,a2e0d5ba-f559-4afc-b1cd-b70a0bb90efa,subscription,0197c9c8-ce12-7529-9800-2852822793a2
2025-07-02 06:18:31.146000+00:00,matthews_correlation_coefficient,0197c9c9-6f6a-79ec-ac1d-a72c8a14a626,0.5105549847284798,None,None,['model_type:original'],quality,0197c9c9-236c-735e-88e2-1eea0acc4032,a2e0d5ba-f559-4afc-b1cd-b70a0bb90efa,subscription,0197c9c8-ce12-7529-9800-2852822793a2
2025-07-02 06:18:31.146000+00:00,f1_measure,0197c9c9-6f6a-79ec-ac1d-a72c8a14a626,0.6666666666666666,None,None,['model_type:original'],quality,0197c9c9-236c-735e-88e2-1eea0acc4032,a2e0d5ba-f559-4afc-b1cd-b70a0bb90efa,subscription,0197c9c8-ce12-7529-9800-2852822793a2
2025-07-02 06:18:31.146000+00:00,accuracy,0197c9c9-6f6a-79ec-ac1d-a72c8a14a626,0.7857142857142857,None,None,['model_type:original'],quality,0197c9c9-236c-735e-88e2-1eea0acc4032,a2e0d5ba-f559-4afc-b1cd-b70a0bb90efa,subscription,0197c9c8-ce12-7529-9800-2852822793a2
2025-07-02 06:18:31.146000+00:00,label_skew,0197c9c9-6f6a-79ec-ac1d-a72c8a14a626,0.6909336273400487,None,None,['model_type:original'],quality,0197c9c9-236c-735e-88e2-1eea0acc4032,a2e0d5ba-f559-4afc-b1cd-b70a0bb90efa,subscription,0197c9c8-ce12-7529-9800-2852822793a2
2025-07-02 06:18:31.146000+00:00,gini_coefficient,0197c9c9-6f6a-79ec-ac1d-a72c8a14a626,0.49790209790209783,None,None,['model_type:original'],quality,0197c9c9-236c-735e-88e2-1eea0acc4032,a2e0d5ba-f559-4afc-b1cd-b70a0bb90efa,subscription,0197c9c8-ce12-7529-9800-2852822793a2
2025-07-02 06:18:31.146000+00:00,log_loss,0197c9c9-6f6a-79ec-ac1d-a72c8a14a626,0.4428034706175754,None,None,['model_type:original'],quality,0197c9c9-236c-735e-88e2-1eea0acc4032,a2e0d5ba-f559-4afc-b1cd-b70a0bb90efa,subscription,0197c9c8-ce12-7529-9800-2852822793a2
2025-07-02 06:18:31.146000+00:00,false_positive_rate,0197c9c9-6f6a-79ec-ac1d-a72c8a14a626,0.13846153846153847,None,None,['model_type:original'],quality,0197c9c9-236c-735e-88e2-1eea0acc4032,a2e0d5ba-f559-4afc-b1cd-b70a0bb90efa,subscription,0197c9c8-ce12-7529-9800-2852822793a2


Note: First 10 records were displayed.
Creating fairness monitor



 Waiting for end of monitor instance creation 0197c9c9-c997-7422-b504-330061224b1a 




active

---------------------------------------
 Monitor instance successfully created 
---------------------------------------


Running fairness monitor



 Waiting for end of monitoring run c806c09e-6957-449f-a09a-ac27a9d63cf6 




error

-------------------------------
 Run failed with status: error 
-------------------------------


Reason: ['code: AIQFM6001, message: Payload logging table rows 20 are less than the minimum rows 100']
Monitor run details: {
    "result": "HTTP response",
    "headers": {
        "_store": {
            "date": [
                "Date",
                "Wed, 02 Jul 2025 06:19:23 GMT"
            ],
            "content-type": [
                "Content-Type",
                "application/json"
            ],
            "transfer-encoding": [
                "Transfer-Encoding",
                "




 Waiting for end of monitor instance creation 0197c9ca-7bb1-7124-b3d9-723eba7cefe1 




preparing...........
active

---------------------------------------
 Monitor instance successfully created 
---------------------------------------


Running drift_v2 monitor



 Waiting for end of monitoring run 7d4afc11-a2e3-4a93-ae5c-bbfb4b6996db 




running..........................................
finished

---------------------------
 Successfully finished run 
---------------------------


Monitor run details: {
    "result": "HTTP response",
    "headers": {
        "_store": {
            "date": [
                "Date",
                "Wed, 02 Jul 2025 06:25:07 GMT"
            ],
            "content-type": [
                "Content-Type",
                "application/json"
            ],
            "transfer-encoding": [
                "Transfer-Encoding",
                "chunked"
            ],
            "connection": [
                "Connection",
                "keep-al

2025-07-02 06:21:36.370920+00:00,records_processed,0197c9cc-42f2-79e2-ba37-96dd0e34f707,20.0,None,None,"['algorithm_used:total_variation', 'computed_on:payload', 'field_type:class', 'field_name:Class Probability for No Risk']",drift_v2,0197c9ca-7bb1-7124-b3d9-723eba7cefe1,7d4afc11-a2e3-4a93-ae5c-bbfb4b6996db,subscription,0197c9c8-ce12-7529-9800-2852822793a2
2025-07-02 06:21:36.370920+00:00,confidence_drift_score,0197c9cc-42f2-79e2-ba37-96dd0e34f707,0.2682,None,0.05,"['algorithm_used:total_variation', 'computed_on:payload', 'field_type:class', 'field_name:Class Probability for No Risk']",drift_v2,0197c9ca-7bb1-7124-b3d9-723eba7cefe1,7d4afc11-a2e3-4a93-ae5c-bbfb4b6996db,subscription,0197c9c8-ce12-7529-9800-2852822793a2
2025-07-02 06:21:36.370920+00:00,records_processed,0197c9cc-42f2-79e2-ba37-96dd0e34f707,20.0,None,None,"['algorithm_used:overlap_coefficient', 'computed_on:payload', 'field_type:class', 'field_name:Class Probability for No Risk']",drift_v2,0197c9ca-7bb1-7124-b3d9-723eba7cefe1,7d4afc11-a2e3-4a93-ae5c-bbfb4b6996db,subscription,0197c9c8-ce12-7529-9800-2852822793a2
2025-07-02 06:21:36.370920+00:00,confidence_drift_score,0197c9cc-42f2-79e2-ba37-96dd0e34f707,0.2692,None,0.05,"['algorithm_used:overlap_coefficient', 'computed_on:payload', 'field_type:class', 'field_name:Class Probability for No Risk']",drift_v2,0197c9ca-7bb1-7124-b3d9-723eba7cefe1,7d4afc11-a2e3-4a93-ae5c-bbfb4b6996db,subscription,0197c9c8-ce12-7529-9800-2852822793a2
2025-07-02 06:21:47.617877+00:00,records_processed,0197c9cc-6ee1-7131-9efb-31483d1f7f8f,20.0,None,None,"['algorithm_used:total_variation', 'computed_on:payload', 'field_type:class', 'field_name:Class Probability for Risk']",drift_v2,0197c9ca-7bb1-7124-b3d9-723eba7cefe1,7d4afc11-a2e3-4a93-ae5c-bbfb4b6996db,subscription,0197c9c8-ce12-7529-9800-2852822793a2
2025-07-02 06:21:47.617877+00:00,confidence_drift_score,0197c9cc-6ee1-7131-9efb-31483d1f7f8f,0.2725,None,0.05,"['algorithm_used:total_variation', 'computed_on:payload', 'field_type:class', 'field_name:Class Probability for Risk']",drift_v2,0197c9ca-7bb1-7124-b3d9-723eba7cefe1,7d4afc11-a2e3-4a93-ae5c-bbfb4b6996db,subscription,0197c9c8-ce12-7529-9800-2852822793a2
2025-07-02 06:21:47.617877+00:00,records_processed,0197c9cc-6ee1-7131-9efb-31483d1f7f8f,20.0,None,None,"['algorithm_used:overlap_coefficient', 'computed_on:payload', 'field_type:class', 'field_name:Class Probability for Risk']",drift_v2,0197c9ca-7bb1-7124-b3d9-723eba7cefe1,7d4afc11-a2e3-4a93-ae5c-bbfb4b6996db,subscription,0197c9c8-ce12-7529-9800-2852822793a2
2025-07-02 06:21:47.617877+00:00,confidence_drift_score,0197c9cc-6ee1-7131-9efb-31483d1f7f8f,0.2735,None,0.05,"['algorithm_used:overlap_coefficient', 'computed_on:payload', 'field_type:class', 'field_name:Class Probability for Risk']",drift_v2,0197c9ca-7bb1-7124-b3d9-723eba7cefe1,7d4afc11-a2e3-4a93-ae5c-bbfb4b6996db,subscription,0197c9c8-ce12-7529-9800-2852822793a2
2025-07-02 06:21:54.678098+00:00,records_processed,0197c9cc-8a76-763d-a26d-5b16403680a3,20.0,None,None,"['algorithm_used:jensen_shannon', 'computed_on:payload', 'field_type:class', 'field_name:prediction']",drift_v2,0197c9ca-7bb1-7124-b3d9-723eba7cefe1,7d4afc11-a2e3-4a93-ae5c-bbfb4b6996db,subscription,0197c9c8-ce12-7529-9800-2852822793a2
2025-07-02 06:21:54.678098+00:00,prediction_drift_score,0197c9cc-8a76-763d-a26d-5b16403680a3,0.1212,None,0.05,"['algorithm_used:jensen_shannon', 'computed_on:payload', 'field_type:class', 'field_name:prediction']",drift_v2,0197c9ca-7bb1-7124-b3d9-723eba7cefe1,7d4afc11-a2e3-4a93-ae5c-bbfb4b6996db,subscription,0197c9c8-ce12-7529-9800-2852822793a2


Note: First 10 records were displayed.
Creating explainability monitor



 Waiting for end of monitor instance creation 0197c9cf-bec9-72c9-84b3-5ab6a3a364c2 




preparing.
active

---------------------------------------
 Monitor instance successfully created 
---------------------------------------


Running explainability monitor for a sample production
Running explanations on scoring IDs: ['0d504f99-968a-4d2f-b1bd-277c3fe6df49-1']
{
  "metadata": {
    "explanation_task_ids": [
      "096f2365-3658-4947-80a8-d1fa522739d8"
    ],
    "created_by": "IBMid-697000GE0W",
    "created_at": "2025-07-02T06:25:42.702984Z"
  }
}
